In [1]:
# Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install lightgbm

In [3]:
# Importing data
df_sales = pd.read_csv("sales_train_evaluation.csv")
sell_prices = pd.read_csv("sell_prices.csv")
calendar = pd.read_csv("calendar.csv")

In [4]:
df_sales.head()

,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_2_001_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,HOBBIES_2_002_CA_3_validation,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
2,HOBBIES_2_003_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,3,1,2,4,0,0,0,1,5,2,1,4,1,0,0,1,2,1,0,1,0,4,1,0,3,0,1,2,2
3,HOBBIES_2_004_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,1,2,0,0,1,0,0,0,...,2,1,1,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0
4,HOBBIES_2_005_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA
0,1/29/2011,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0
1,1/30/2011,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0
2,1/31/2011,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0
3,2/1/2011,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1
4,2/2/2011,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1


In [6]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,HOBBIES_2_001,11105,5.47
1,CA_3,HOBBIES_2_001,11106,5.47
2,CA_3,HOBBIES_2_001,11107,5.47
3,CA_3,HOBBIES_2_001,11108,5.47
4,CA_3,HOBBIES_2_001,11109,5.47


In [7]:
import warnings 
warnings.filterwarnings('ignore')

In [8]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [9]:
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)

sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)

sales_train_validation = pd.read_csv('sales_train_evaluation.csv')

submission = pd.read_csv('sample_submission.csv')
submission = reduce_mem_usage(submission)

Mem. usage decreased to  0.12 Mb (34.4% reduction)
Mem. usage decreased to  0.62 Mb (37.5% reduction)
Mem. usage decreased to  0.01 Mb (84.2% reduction)


In [10]:
sales_train_validation = pd.read_csv("sales_train_evaluation.csv")

In [11]:
sales_train_validation.head()

,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_2_001_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,HOBBIES_2_002_CA_3_validation,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
2,HOBBIES_2_003_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,3,1,2,4,0,0,0,1,5,2,1,4,1,0,0,1,2,1,0,1,0,4,1,0,3,0,1,2,2
3,HOBBIES_2_004_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,1,2,0,0,1,0,0,0,...,2,1,1,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0
4,HOBBIES_2_005_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
sales_train_validation = pd.melt(sales_train_validation,
                                 id_vars = ['id'], 
                                 var_name = 'day',
                                 value_name = 'demand')
sales_train_validation = reduce_mem_usage(sales_train_validation)

Mem. usage decreased to  4.69 Mb (29.2% reduction)


In [13]:
print(sales_train_validation)

                                   id     day  demand
0       HOBBIES_2_001_CA_3_validation     d_1       0
1       HOBBIES_2_002_CA_3_validation     d_1       0
2       HOBBIES_2_003_CA_3_validation     d_1       0
3       HOBBIES_2_004_CA_3_validation     d_1       0
4       HOBBIES_2_005_CA_3_validation     d_1       0
...                               ...     ...     ...
289204  HOBBIES_2_145_CA_3_validation  d_1941       0
289205  HOBBIES_2_146_CA_3_validation  d_1941       0
289206  HOBBIES_2_147_CA_3_validation  d_1941       0
289207  HOBBIES_2_148_CA_3_validation  d_1941       1
289208  HOBBIES_2_149_CA_3_validation  d_1941       0

[289209 rows x 3 columns]


In [14]:
print(submission.head())

                              id  F1  F2  F3  F4  ...  F24  F25  F26  F27  F28
0  HOBBIES_2_001_CA_3_validation   0   0   0   0  ...    0    0    0    0    0
1  HOBBIES_2_002_CA_3_validation   0   0   0   0  ...    0    0    0    0    0
2  HOBBIES_2_003_CA_3_validation   0   0   0   0  ...    0    0    0    0    0
3  HOBBIES_2_004_CA_3_validation   0   0   0   0  ...    0    0    0    0    0
4  HOBBIES_2_005_CA_3_validation   0   0   0   0  ...    0    0    0    0    0

[5 rows x 29 columns]


In [15]:
print(len(submission))

149


In [16]:
test2_rows = [row for row in submission['id'] if 'validation' in row]
test2 = submission[submission['id'].isin(test2_rows)]

test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                  'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

In [17]:
test2.head()

,id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_2_001_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_2_002_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_2_003_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_2_004_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_2_005_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
product = sales_train_validation['id'].drop_duplicates()

In [19]:
test2 = test2.merge(product, how = 'left', on = 'id')
test2

,id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_2_001_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_2_002_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_2_003_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_2_004_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_2_005_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,HOBBIES_2_145_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
145,HOBBIES_2_146_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
146,HOBBIES_2_147_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
147,HOBBIES_2_148_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
test2 = pd.melt(test2, 
                id_vars = ['id'], 
                var_name = 'day', 
                value_name = 'demand')
test2

,id,day,demand
0,HOBBIES_2_001_CA_3_validation,d_1942,0
1,HOBBIES_2_002_CA_3_validation,d_1942,0
2,HOBBIES_2_003_CA_3_validation,d_1942,0
3,HOBBIES_2_004_CA_3_validation,d_1942,0
4,HOBBIES_2_005_CA_3_validation,d_1942,0
...,...,...,...
4167,HOBBIES_2_145_CA_3_validation,d_1969,0
4168,HOBBIES_2_146_CA_3_validation,d_1969,0
4169,HOBBIES_2_147_CA_3_validation,d_1969,0
4170,HOBBIES_2_148_CA_3_validation,d_1969,0


In [21]:
sales_train_validation['part'] = 'train'
test2['part'] = 'test'

In [22]:
sales_train_validation

,id,day,demand,part
0,HOBBIES_2_001_CA_3_validation,d_1,0,train
1,HOBBIES_2_002_CA_3_validation,d_1,0,train
2,HOBBIES_2_003_CA_3_validation,d_1,0,train
3,HOBBIES_2_004_CA_3_validation,d_1,0,train
4,HOBBIES_2_005_CA_3_validation,d_1,0,train
...,...,...,...,...
289204,HOBBIES_2_145_CA_3_validation,d_1941,0,train
289205,HOBBIES_2_146_CA_3_validation,d_1941,0,train
289206,HOBBIES_2_147_CA_3_validation,d_1941,0,train
289207,HOBBIES_2_148_CA_3_validation,d_1941,1,train


In [23]:
data = pd.concat([sales_train_validation, test2], axis = 0)
import gc
del sales_train_validation, test2
gc.collect()

95

In [24]:
data

,id,day,demand,part
0,HOBBIES_2_001_CA_3_validation,d_1,0,train
1,HOBBIES_2_002_CA_3_validation,d_1,0,train
2,HOBBIES_2_003_CA_3_validation,d_1,0,train
3,HOBBIES_2_004_CA_3_validation,d_1,0,train
4,HOBBIES_2_005_CA_3_validation,d_1,0,train
...,...,...,...,...
4167,HOBBIES_2_145_CA_3_validation,d_1969,0,test
4168,HOBBIES_2_146_CA_3_validation,d_1969,0,test
4169,HOBBIES_2_147_CA_3_validation,d_1969,0,test
4170,HOBBIES_2_148_CA_3_validation,d_1969,0,test


In [25]:
data.reset_index(drop=True,inplace=True)
# data = data.loc[30000000:]
calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)

In [26]:
data['item_id'] = data['id']
data

,id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id
0,HOBBIES_2_001_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_001_CA_3_validation
1,HOBBIES_2_002_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_002_CA_3_validation
2,HOBBIES_2_003_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_003_CA_3_validation
3,HOBBIES_2_004_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_004_CA_3_validation
4,HOBBIES_2_005_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_005_CA_3_validation
...,...,...,...,...,...,...,...,...,...,...,...
293376,HOBBIES_2_145_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_145_CA_3_validation
293377,HOBBIES_2_146_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_146_CA_3_validation
293378,HOBBIES_2_147_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_147_CA_3_validation
293379,HOBBIES_2_148_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_148_CA_3_validation


In [27]:
for i in range(0, len(data['id'])):
    keep = data['id'][i]
    keep2 = keep[0:13]
    data['item_id'][i] = keep2

In [28]:
data

,id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id
0,HOBBIES_2_001_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_001
1,HOBBIES_2_002_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_002
2,HOBBIES_2_003_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_003
3,HOBBIES_2_004_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_004
4,HOBBIES_2_005_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_005
...,...,...,...,...,...,...,...,...,...,...,...
293376,HOBBIES_2_145_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_145
293377,HOBBIES_2_146_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_146
293378,HOBBIES_2_147_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_147
293379,HOBBIES_2_148_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_148


In [29]:
sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,HOBBIES_2_001,11105,5.468750
1,CA_3,HOBBIES_2_001,11106,5.468750
2,CA_3,HOBBIES_2_001,11107,5.468750
3,CA_3,HOBBIES_2_001,11108,5.468750
4,CA_3,HOBBIES_2_001,11109,5.468750
...,...,...,...,...
32493,CA_3,HOBBIES_2_149,11617,0.970215
32494,CA_3,HOBBIES_2_149,11618,0.970215
32495,CA_3,HOBBIES_2_149,11619,0.970215
32496,CA_3,HOBBIES_2_149,11620,0.970215


In [30]:
data = data.merge(sell_prices, on = ['item_id', 'wm_yr_wk'], how = 'left')
del calendar,sell_prices,product
gc.collect()

24

In [31]:
data

,id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id,store_id,sell_price
0,HOBBIES_2_001_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_001,NaN,NaN
1,HOBBIES_2_002_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_002,CA_3,1.969727
2,HOBBIES_2_003_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_003,NaN,NaN
3,HOBBIES_2_004_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_004,NaN,NaN
4,HOBBIES_2_005_CA_3_validation,0,train,1/29/2011,11101,NaN,NaN,NaN,NaN,0,HOBBIES_2_005,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293376,HOBBIES_2_145_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_145,CA_3,2.880859
293377,HOBBIES_2_146_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_146,CA_3,1.969727
293378,HOBBIES_2_147_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_147,CA_3,0.970215
293379,HOBBIES_2_148_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_148,CA_3,0.879883


In [32]:
from sklearn.preprocessing import LabelEncoder
nan_features = ['event_type_1', 'event_type_2', 'event_name_1', 'event_name_2'] # 
for feature in nan_features:
    data[feature].fillna('unknown', inplace = True)

In [33]:
data

,id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id,store_id,sell_price
0,HOBBIES_2_001_CA_3_validation,0,train,1/29/2011,11101,unknown,unknown,unknown,unknown,0,HOBBIES_2_001,NaN,NaN
1,HOBBIES_2_002_CA_3_validation,0,train,1/29/2011,11101,unknown,unknown,unknown,unknown,0,HOBBIES_2_002,CA_3,1.969727
2,HOBBIES_2_003_CA_3_validation,0,train,1/29/2011,11101,unknown,unknown,unknown,unknown,0,HOBBIES_2_003,NaN,NaN
3,HOBBIES_2_004_CA_3_validation,0,train,1/29/2011,11101,unknown,unknown,unknown,unknown,0,HOBBIES_2_004,NaN,NaN
4,HOBBIES_2_005_CA_3_validation,0,train,1/29/2011,11101,unknown,unknown,unknown,unknown,0,HOBBIES_2_005,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293376,HOBBIES_2_145_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_145,CA_3,2.880859
293377,HOBBIES_2_146_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_146,CA_3,1.969727
293378,HOBBIES_2_147_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_147,CA_3,0.970215
293379,HOBBIES_2_148_CA_3_validation,0,test,6/19/2016,11621,NBAFinalsEnd,Sporting,Father's day,Cultural,0,HOBBIES_2_148,CA_3,0.879883


In [34]:
cat = ['item_id', 'event_type_1', 'event_type_2','event_name_2','event_name_1'] 
for feature in cat:
    encoder = LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature])

In [35]:
data

,id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id,store_id,sell_price
0,HOBBIES_2_001_CA_3_validation,0,train,1/29/2011,11101,30,4,4,2,0,0,NaN,NaN
1,HOBBIES_2_002_CA_3_validation,0,train,1/29/2011,11101,30,4,4,2,0,1,CA_3,1.969727
2,HOBBIES_2_003_CA_3_validation,0,train,1/29/2011,11101,30,4,4,2,0,2,NaN,NaN
3,HOBBIES_2_004_CA_3_validation,0,train,1/29/2011,11101,30,4,4,2,0,3,NaN,NaN
4,HOBBIES_2_005_CA_3_validation,0,train,1/29/2011,11101,30,4,4,2,0,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
293376,HOBBIES_2_145_CA_3_validation,0,test,6/19/2016,11621,16,3,2,0,0,144,CA_3,2.880859
293377,HOBBIES_2_146_CA_3_validation,0,test,6/19/2016,11621,16,3,2,0,0,145,CA_3,1.969727
293378,HOBBIES_2_147_CA_3_validation,0,test,6/19/2016,11621,16,3,2,0,0,146,CA_3,0.970215
293379,HOBBIES_2_148_CA_3_validation,0,test,6/19/2016,11621,16,3,2,0,0,147,CA_3,0.879883


In [36]:
# Creating the time lagged features
data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
data['rolling_mean_t7']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
data['rolling_std_t7']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
data['rolling_mean_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())

In [37]:
data = reduce_mem_usage(data)

Mem. usage decreased to 17.91 Mb (52.2% reduction)


In [38]:
data['rolling_mean_t90']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
data['rolling_std_t30']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
data['rolling_skew_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
data['rolling_kurt_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())

In [39]:
data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)

In [40]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['week'] = data['date'].dt.week
data['day'] = data['date'].dt.day
data['dayofweek'] = data['date'].dt.dayofweek

In [41]:
def weekend(arg):
    if arg==5 or arg==6:
        return 1
    else:
        return 0
data['isweekend'] = data['dayofweek'].apply(weekend)

In [42]:
data['revenue'] = data['demand'] * data['sell_price']
data['lag_revenue_t1'] = data.groupby(['id'])['revenue'].transform(lambda x: x.shift(28))
data['rolling_revenue_std_t28'] = data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).std())
data['rolling_revenue_mean_t28'] = data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).mean())
data.drop(['revenue'],axis=1,inplace=True)

In [43]:
features = ['item_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_type_1', 'event_type_2', 
            'snap_CA', 'sell_price', 'lag_t28','lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30',
            'isweekend','lag_revenue_t1','rolling_revenue_std_t28','rolling_revenue_mean_t28', 'event_name_2', 'event_name_1'] #,
data = reduce_mem_usage(data)

Mem. usage decreased to 26.58 Mb (42.8% reduction)


In [44]:
data['dept_id'] = 'Hobbies'
data['store_id'] = 'CA_3'
data

,id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,item_id,store_id,sell_price,lag_t28,lag_t29,lag_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,month,week,day,dayofweek,isweekend,lag_revenue_t1,rolling_revenue_std_t28,rolling_revenue_mean_t28,dept_id
0,HOBBIES_2_001_CA_3_validation,0,train,2011-01-29,11101,30,4,4,2,0,0,CA_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,1,4,29,5,1,NaN,NaN,NaN,Hobbies
1,HOBBIES_2_002_CA_3_validation,0,train,2011-01-29,11101,30,4,4,2,0,1,CA_3,1.969727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,1,4,29,5,1,NaN,NaN,NaN,Hobbies
2,HOBBIES_2_003_CA_3_validation,0,train,2011-01-29,11101,30,4,4,2,0,2,CA_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,1,4,29,5,1,NaN,NaN,NaN,Hobbies
3,HOBBIES_2_004_CA_3_validation,0,train,2011-01-29,11101,30,4,4,2,0,3,CA_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,1,4,29,5,1,NaN,NaN,NaN,Hobbies
4,HOBBIES_2_005_CA_3_validation,0,train,2011-01-29,11101,30,4,4,2,0,4,CA_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011,1,4,29,5,1,NaN,NaN,NaN,Hobbies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293376,HOBBIES_2_145_CA_3_validation,0,test,2016-06-19,11621,16,3,2,0,0,144,CA_3,2.880859,0.0,2.0,0.0,0.428467,0.786621,0.766602,0.644531,0.549805,1.304688,1.762695,1.957031,0.0,0.0,0.0,0.0,2016,6,24,19,6,1,0.000000,3.845703,2.367188,Hobbies
293377,HOBBIES_2_146_CA_3_validation,0,test,2016-06-19,11621,16,3,2,0,0,145,CA_3,1.969727,0.0,0.0,0.0,0.000000,0.000000,0.366699,0.222168,0.205566,0.808594,2.140625,3.623047,0.0,0.0,0.0,0.0,2016,6,24,19,6,1,0.000000,1.297852,0.562988,Hobbies
293378,HOBBIES_2_147_CA_3_validation,0,test,2016-06-19,11621,16,3,2,0,0,146,CA_3,0.970215,0.0,0.0,0.0,0.714355,1.889648,0.700195,0.455566,0.461182,1.149414,2.246094,5.972656,0.0,0.0,0.0,0.0,2016,6,24,19,6,1,0.000000,1.139648,0.727539,Hobbies
293379,HOBBIES_2_148_CA_3_validation,0,test,2016-06-19,11621,16,3,2,0,0,147,CA_3,0.879883,1.0,1.0,0.0,0.428467,0.534668,0.366699,0.233276,0.411133,0.999512,3.835938,16.609375,0.0,0.0,0.0,0.0,2016,6,24,19,6,1,0.879883,0.906738,0.345703,Hobbies


In [45]:
# RUN VANAF HIER OM PARAMETERS TE TUNEN

In [379]:
x_train = data[data['part'] == 'train']
y_train = x_train['demand']
x_val = data[(data['date'] > '2016-04-24') & (data['part'] == 'train')]
y_val = x_val['demand']
test = data[data['date'] > '2016-04-24']
test.loc[test['part']=='train','id'] = test.loc[test['part']=='train','id'].str.replace('_evaluation','_validation')
x_train.drop(['demand','part',],inplace=True,axis=1)
x_val.drop(['demand','part',],inplace=True,axis=1)
test.drop(['demand','part',],inplace=True,axis=1)

In [380]:
from lightgbm import Dataset,train,plot_importance
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'lambda_l1': 3.3,         
#    'min_gain_to_split': 50, #-> niet bruikbaar
#    'num_leaves': 64,       #KAN INTERESSANT ZIJN ICM MAX DEPTH (64)
#    ' bagging_freq': 2,
#   'bagging_fraction': 0.9, #geen resultaat
 #   'max_bin': 100,        #default  = 255 -> gebeurt weinig
 #   'verbose': 50,          #default is 1 -> doet niks
#    'feature_fraction': 0.999, #zorgt voor hogere rmse
 #   'lambda_l2': 1,    #hogere rmse
 #   'n_jobs': -1,
 #   'seed': 236,
#   'max_depth': 8,        #default is -1 # KAN INTERESSANT ZIJN ICM NUM_LEAVES (8)
    'learning_rate': 0.3}   #stond eerst op 0.1

In [381]:
category = x_train['dept_id'].unique()
category2 = x_train['store_id'].unique()
def dataset(categor,categor2):
    tindex = x_train[(x_train['dept_id']==categor) & (x_train['store_id']==categor2)].index.values
    vindex = x_val[(x_val['dept_id']==categor)&(x_val['store_id']==categor2)].index.values
    
    x_t,x_v,y_t,y_v,t =  x_train[(x_train['dept_id']==categor) & (x_train['store_id']==categor2)],x_val[(x_val['dept_id']==categor)&(x_val['store_id']==categor2)],y_train.loc[tindex],y_val.loc[vindex],test[(test['dept_id']==categor) &(test['store_id']==categor2)]
    
    x_train.drop(tindex,axis=0,inplace=True)
    x_val.drop(vindex,axis=0,inplace=True)
    test.drop(test[(test['dept_id']==categor) &(test['store_id']==categor2)].index.values,axis=0,inplace=True)
    
    return x_t,x_v,y_t,y_v,t

In [382]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
TEST = None


for i in category:
    for j in category2:
        x_t,x_v,y_t,y_v,t = dataset(i,j)
        train_set = Dataset(x_t[features], y_t)
        val_set = Dataset(x_v[features], y_v)
        del x_t, y_t
        gc.collect()

        model = train(params, train_set, num_boost_round = 10, early_stopping_rounds = 40, valid_sets = [train_set, val_set], verbose_eval = 2) #
        y_pred = model.predict(t[features])

        t['demand'] = y_pred

        TEST = pd.concat([TEST,t],axis=0)
        del x_v,y_v,t,train_set,val_set,y_pred,model
        gc.collect()

Training until validation scores don't improve for 40 rounds.
[2]	training's rmse: 0.850751	valid_1's rmse: 1.16329
[4]	training's rmse: 0.829696	valid_1's rmse: 1.13395
[6]	training's rmse: 0.81953	valid_1's rmse: 1.12339
[8]	training's rmse: 0.811868	valid_1's rmse: 1.10149
[10]	training's rmse: 0.80675	valid_1's rmse: 1.09865
Did not meet early stopping. Best iteration is:
[10]	training's rmse: 0.80675	valid_1's rmse: 1.09865


In [383]:
predictions1 = TEST[TEST['id'].apply(lambda x: "validation" in x)][['id', 'date', 'demand']]
predictions2 = TEST[TEST['id'].apply(lambda x: "evaluation" in x)][['id', 'date', 'demand']]

In [384]:
prediction1 = pd.pivot(predictions1, index = 'id', columns = 'date', values = 'demand').reset_index()
prediction2 = pd.pivot(predictions2, index = 'id', columns = 'date', values = 'demand').reset_index()

In [385]:
predictions = pd.concat([prediction1,prediction2],axis=0)
predictions.to_csv("submissionstoreanddeptwiseGBM_FINAL.csv",index=False)